In [1]:
import torch
from torch.utils.data import DataLoader
from state_quantization.dataset import load_dataset
from state_quantization.dataset import DynamicsModelDataset
from state_quantization.quantization_models import ForcastingDiscHC, ForcastingDiscHCConst
from state_quantization.train import train_model, test_step
from state_quantization.eval import eval_model, compare_models

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
print(f"Using Device: {device}")
torch.backends.cudnn.benchmark = True


Using Device: cuda:0


## Load Dataset

In [3]:
dataset_input_key = 'merged_input'
dataset_output_key = 'merged_output'
dataset_file_path = 'tmp/ib-out/ib-samples-la.npy'
normalized_data_params_save_path = 'state_quantization/NormalizeInputConfigs.pkl'
dataset_device = 'cpu'
y_indexes = [4, 6]

train_dataset, val_dataset = load_dataset(file_path=dataset_file_path, input_key=dataset_input_key,
                                          output_key=dataset_output_key, dataset_class=DynamicsModelDataset,
                                          normalize=True, device=dataset_device, y_clip_range=y_indexes,
                                          normalized_data_params_save_path=normalized_data_params_save_path)

batch_size = 500
params = {'batch_size': batch_size,
          'shuffle': True,
          'num_workers': 0,
          'drop_last': True,
          'pin_memory': not train_dataset.x.is_cuda}
print(params)
train_loader = DataLoader(train_dataset, **params)
val_loader = DataLoader(val_dataset, **params)
train_dataset.y[0]

Clipping y
torch.Size([672000, 39, 6])
torch.Size([672000, 10, 2])
torch.Size([288000, 39, 6])
torch.Size([288000, 10, 2])
{'batch_size': 500, 'shuffle': True, 'num_workers': 0, 'drop_last': True, 'pin_memory': True}


tensor([[-0.2227, -0.6698],
        [-0.3738, -0.6790],
        [-0.5700, -0.7000],
        [-0.3641, -0.6819],
        [-0.5543, -0.6388],
        [-0.7021, -0.6682],
        [-0.3377, -0.6477],
        [-0.6460, -0.5929],
        [-0.4730, -0.5331],
        [-0.4207, -0.4667]])

## Define model

In [4]:
model_dict_path = 'state_quantization/model_dict'
model_path = 'state_quantization/model'
num_of_features = train_dataset.get_features_size()
seq_len = train_dataset.get_seq_len()
hidden_size = 16
out_size = train_dataset.get_output_feature_size()
print(f'Out Size:{out_size}')
look_ahead = train_dataset.get_look_ahead_size()
n_layers = 1
dropout_p = 0.2
print(look_ahead)
model = ForcastingDiscHC(features=num_of_features, hidden_size=hidden_size, out_size=out_size, seq_len=seq_len,
                               n_layers=n_layers, dropout=dropout_p,
                               look_ahead=look_ahead).to(device=device)

load_to_gpu = model.is_cuda() and not train_dataset.x.is_cuda
print(load_to_gpu)

Out Size:2
9
LSTM Layers
ModuleList(
  (0): LSTMCell(6, 16)
)
LSTM Dropout Layers
ModuleList()
Fully Connected Layers
ModuleList(
  (0): Linear(in_features=16, out_features=16, bias=True)
  (1): GELU(approximate=none)
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=16, out_features=16, bias=True)
  (4): GELU(approximate=none)
  (5): Dropout(p=0.2, inplace=False)
  (6): Linear(in_features=16, out_features=2, bias=True)
)
True


## Train Model

In [ ]:
train_model(model=model, train_loader=train_loader, test_loader=val_loader, n_epochs=25, learning_rate=1e-3,
            load_to_gpu=load_to_gpu, gamma=0.1, lr_milestones=[4, 9])

Untrained test
--------
Test loss: 1.0199950858950615

Epoch 0
---------
Train loss: 0.3326351976332565
Test loss: 0.20933338226233092
lr: [0.001]
Epoch time: epoch_time = 34.096s
Epoch 1
---------
Train loss: 0.25464472519455567
Test loss: 0.19997173501178622
lr: [0.001]
Epoch time: epoch_time = 33.299s
Epoch 2
---------
Train loss: 0.25867640677218634
Test loss: 0.2048993517075562
lr: [0.001]
Epoch time: epoch_time = 34.605s
Epoch 3
---------
Train loss: 0.2544462426366018
Test loss: 0.1981309620767004
lr: [0.0001]
Epoch time: epoch_time = 34.368s
Epoch 4
---------
Train loss: 0.2540367470522012
Test loss: 0.19503884120947784
lr: [0.0001]
Epoch time: epoch_time = 34.353s
Epoch 5
---------
Train loss: 0.2543599897596453
Test loss: 0.19358572436289656
lr: [0.0001]
Epoch time: epoch_time = 34.414s
Epoch 6
---------
Train loss: 0.25449310128931846
Test loss: 0.1912616823748168
lr: [0.0001]
Epoch time: epoch_time = 34.837s
Epoch 7
---------
Train loss: 0.25293993542990867
Test loss: 0.192

## Save Model

In [ ]:

torch.save(model.state_dict(), model_dict_path)
torch.save(model, model_path)
torch.cuda.empty_cache()
del model

## Load Model for evaluation

In [ ]:

model_classes = [ForcastingDiscHC, ForcastingDiscHCConst]
models = []
for model_class in model_classes:
    models.append(model_class(features=num_of_features, hidden_size=hidden_size, out_size=out_size,
                              seq_len=seq_len, n_layers=n_layers, dropout_p=dropout_p,
                              look_ahead=look_ahead).to(device))
    models[-1].load_state_dict(torch.load(model_dict_path))

In [ ]:
for la_model in models:
    print(la_model.__class__.__name__)
    test_step(model=la_model, data_loader=val_loader, loss_function=torch.nn.MSELoss(), load_to_gpu=load_to_gpu)

In [ ]:
#value_keys = ['setpoint', 'velocity', 'gain', 'shift', 'fatigue', 'consumption']
value_keys = ['fatigue', 'consumption']

models = []
for model_class in model_classes:
    models.append(model_class(features=num_of_features, hidden_size=hidden_size, out_size=out_size,
                              seq_len=seq_len, n_layers=n_layers, dropout_p=dropout_p,
                              look_ahead=look_ahead).to(dataset_device))
    models[-1].load_state_dict(torch.load(model_dict_path))

compare_models(models, x=val_dataset.x[78], y=val_dataset.y[78], value_keys=value_keys)